# Домашнее задание 4

## Глубинное обучение в анализе графовых данных, ПМИ ВШЭ

In [1]:
import torch

# !pip uninstall torch-scatter torch-sparse torch-geometric torch-cluster  --y
# !pip install torch-scatter -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-sparse -f https://data.pyg.org/whl/torch-{torch.__version__}.html
# !pip install torch-cluster -f https://data.pyg.org/whl/torch-{torch.__version__}.html
!pip install git+https://github.com/pyg-team/pytorch_geometric.git

  Cloning https://github.com/pyg-team/pytorch_geometric.git to /tmp/pip-req-build-aio9909s
  Running command git clone --filter=blob:none --quiet https://github.com/pyg-team/pytorch_geometric.git /tmp/pip-req-build-aio9909s
  Resolved https://github.com/pyg-team/pytorch_geometric.git to commit 11a29b097a7448b76375045e753154f79f4e5bd2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


## 1. Реализация TransE (10 баллов)

В этом задании требуется реализовать пайплайн обучения эмбдеддингов графа знаний с помощью [TransE](https://proceedings.neurips.cc/paper/2013/file/1cecc7a77928ca8133fa24680a88d2f9-Paper.pdf) для задачи прогнозирования отсутствующих ребер на наборе данных [Freebase](https://paperswithcode.com/dataset/fb15k) (FB15k-237), а также реализовать саму модель.

In [2]:
import torch_geometric
from torch_geometric.datasets.rel_link_pred_dataset import RelLinkPredDataset


dataset = RelLinkPredDataset('data', 'FB15k-237')
data = dataset[0]

#### TransE
Ребра в графе знаний представляются тройками $(h, r, t)$. В TransE мы моделируем как объекты, так и отношения в пространстве эмбеддингов и пытаемся получить эмбеддинги, как $\textbf h + \textbf l \approx \textbf t$. Формально loss выглядит:

$$\sum_{((h, l, t), (h', l, t')) \in T_{batch}} [\gamma + d(\textbf{h} + \textbf{l}, \textbf t) - d(\textbf{h'} + \textbf l, \textbf{t'})]$$

где $(h', l, t')$ представляет собой тройку, заменяя head или tail случайным объектом.
$d(\boldsymbol{h}+\boldsymbol{l}, \boldsymbol{t})$ – показатель _различия_ положительного ребра. Кроме того, $d\left(\boldsymbol{h}^{\prime}+\boldsymbol{l}, \boldsymbol{t}^{\prime}\right)$ — это оценка _различия_ для отрицательной тройки, полученная изменением либо head, либо tail (но не оба) положительной тройки. Таким образом, TransE *предпочитает* более низкие оценки для положительных ребер и большие оценки для отрицательных ребер.

Что касается параметра $\gamma$, он используется для обеспечения того, чтобы оценка положительного ребра отличалась от оценки отрицательного ребра как минимум на $\gamma$.

Итого алгоритм TransE выглядит следующим образом:

![](https://production-media.paperswithcode.com/methods/Screen_Shot_2020-05-27_at_12.01.23_AM.png)

Что касается реализации модели, можно инициализировать $\textbf l$ и $\textbf e$ в соответствии с приведенным выше псевдокодом. Чтобы вычислить $d(\textbf{h} + \textbf{l}, \textbf t)$, нужно взять L2-норму $\textbf h + \textbf l - \textbf t$.

*Примечание: для повышения производительности можно нормализовать $\textbf e$ каждую эпоху, а не каждый мини-батч.*

__Вспомогательные функции:__

Одним из ключевых аспектов обучения модели является создание измененных троек путем замены head или tail случайным объектом:

In [3]:
def create_neg_edge_index(edge_index, edge_type, num_entities):
    head_or_tail = torch.randint(high=2, size=edge_type.size(),
                                 device=device)
    rand_entities = torch.randint(high=num_entities,
                                  size=edge_type.size(), device=device)
    # change when 1, otherwise regular head
    heads = torch.where(head_or_tail == 1, rand_entities,
                        edge_index[0, :])
    # change when 0, otherwise regular tail
    tails = torch.where(head_or_tail == 0, rand_entities,
                        edge_index[1, :])
    return torch.stack([heads, tails], dim=0)

Оценивать качество будем по Hits@10, Mean Rank и MRR (mean reciprocal rank).

Hits@10 = $\frac{|\{r \in P | r \leq 10\}|}{|P|}$, где $|P|$ — количество оценок, а $r$ — ранг.

Mean Rank = $\frac{1}{|P|}\sum_{r \in P}r$

MMR = $\frac{1}{|P|}\sum_{r \in P}\frac{1}{r}$

Подробнее о метриках можно узнать [здесь](https://arxiv.org/pdf/2002.06914.pdf).

In [4]:
def mrr(predictions, gt):
    indices = predictions.argsort()
    return (1.0 / (indices == gt).nonzero()[:, 1].float().add(1.0)).sum().item()


def mr(predictions, gt):
    indices = predictions.argsort()
    return ((indices == gt).nonzero()[:, 1].float().add(1.0)).sum().item()


def hit_at_k(predictions, gt, device, k=10):
    zero_tensor = torch.tensor([0], device=device)
    one_tensor = torch.tensor([1], device=device)
    _, indices = predictions.topk(k=k, largest=False)
    return torch.where(indices == gt, one_tensor, zero_tensor).sum().item()

__Требуется__ добиться качества хотя бы 0.17 MRR и 0.30 Hits@10

In [5]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm
from torch.utils import data

data_path = './data/FB15K'
train_batch_size = 1024
eval_batch_size = 512
epochs = 100 # 2000
learning_rate = 0.1

hidden_size = 50
eval_freq = 25
margin = 1.
seed = 3435

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
def load_dict(path) :
    output_dict = {}

    for line in open(path, 'r').readlines()[1:] :
        key, value = line.strip().split('\t')
        output_dict[key] = int(value)

    return output_dict

class FB15K_Dataset(data.Dataset) :
    def __init__(self, path, entity2id, relation2id):
        self.entity2id = entity2id
        self.relation2id = relation2id

        self.data = []
        for line in open(path, 'r').readlines()[1:]:
            self.data.append(list(map(int, line.strip().split())))

    def __len__(self):
        return len(self.data)

    def __getitem__(self, item):
        return self.data[item]

In [7]:
entity2id = load_dict(os.path.join(data_path, 'entity2id.txt'))
relation2id = load_dict(os.path.join(data_path, 'relation2id.txt'))

train_dataset = FB15K_Dataset(os.path.join(data_path, 'train2id.txt'), entity2id, relation2id)
dev_dataset = FB15K_Dataset(os.path.join(data_path, 'valid2id.txt'), entity2id, relation2id)
test_dataset = FB15K_Dataset(os.path.join(data_path, 'test2id.txt'), entity2id, relation2id)

train_loader = data.DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
dev_loader = data.DataLoader(dev_dataset, batch_size=eval_batch_size, shuffle=False)
test_loader = data.DataLoader(test_dataset, batch_size=eval_batch_size, shuffle=False)

In [8]:
class TransE(nn.Module):
    def __init__(self, n_entity, n_relation, hidden_size, margin=1.0, device=True):
        super(TransE, self).__init__()
        self.device = device

        self.n_entity = n_entity
        self.n_relation = n_relation
        self.hidden_size = hidden_size

        self.entity_embedding = nn.Embedding(self.n_entity + 1, self.hidden_size, padding_idx=self.n_entity)
        self.relation_embedding = nn.Embedding(self.n_relation + 1, self.hidden_size, padding_idx=self.n_relation)

        self.init_weight(self.entity_embedding)
        self.init_weight(self.relation_embedding)

        self.loss_func = nn.MarginRankingLoss(margin=margin, reduction='none')

    def init_weight(self, embedding):
        n_vocab, hidden_dim = embedding.weight.data.size()
        sqrt_dim = hidden_dim ** 0.5

        embedding.weight.data = torch.FloatTensor(n_vocab, hidden_dim).uniform_(-6./sqrt_dim, 6./sqrt_dim)
        embedding.weight.data = F.normalize(embedding.weight.data, 2, 1)

    def get_score(self, triple):
        sbj, rel, obj = triple[:, 0], triple[:, 1], triple[:, 2]

        sbj_embedding = self.entity_embedding(sbj)
        rel_embedding = self.relation_embedding(rel)
        obj_embedding = self.entity_embedding(obj)

        score = torch.norm((sbj_embedding + rel_embedding - obj_embedding), p=1, dim=1)

        return score

    def forward(self, positive_triple, negative_triple):
        positive_score = self.get_score(positive_triple)
        negative_score = self.get_score(negative_triple)

        y = torch.tensor([-1.], dtype=torch.float, device=self.device)

        return self.loss_func(positive_score, negative_score, y)

In [9]:
def evaluation(model, data_loader, device) :
    model.eval()
    hit_at_10, mrr_, total = 0., 0., 0.

    entity_ids = torch.arange(model.n_entity, device=device).unsqueeze(0)

    for sbj, obj, rel in data_loader :
        sbj, rel, obj = sbj.to(device), rel.to(device), obj.to(device)
        b_size = sbj.size(0)
        all_entity = entity_ids.repeat(b_size, 1)
        repeat_sbj = sbj.unsqueeze(1).repeat(1, all_entity.size(1))
        repeat_rel = rel.unsqueeze(1).repeat(1, all_entity.size(1))
        repeat_obj = obj.unsqueeze(1).repeat(1, all_entity.size(1))

        sbj_triples = torch.stack((repeat_sbj, repeat_rel, all_entity), dim=2).view(-1, 3)
        obj_triples = torch.stack((all_entity, repeat_rel, repeat_obj), dim=2).view(-1, 3)

        obj_pred_score = model.get_score(sbj_triples).view(b_size, -1)
        sbj_pred_score = model.get_score(obj_triples).view(b_size, -1)

        pred = torch.cat([sbj_pred_score, obj_pred_score], dim=0)
        answer = torch.cat([sbj, obj], dim=0)

        hit_at_10 += hit_at_k(pred, answer, device, k=10)

        mrr_ += mrr(pred, answer)
        total += pred.size(0)

    hit_at_10_score = hit_at_10 / total * 100.
    mrr_score = mrr_ / total * 100.

    return hit_at_10_score, mrr_score

In [10]:
model = TransE(n_entity=len(entity2id), n_relation=len(relation2id), hidden_size=hidden_size, margin=margin, device=device)
model.to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

print('Model Structure : {}'.format(model))

Model Structure : TransE(
  (entity_embedding): Embedding(14952, 50, padding_idx=14951)
  (relation_embedding): Embedding(1346, 50, padding_idx=1345)
  (loss_func): MarginRankingLoss()
)


In [11]:
best_score = 0.
for epoch in range(1, epochs+1) :
    model.train()
    for i, (sbj, obj, rel) in enumerate(train_loader) :
        sbj, rel, obj = sbj.to(device), rel.to(device), obj.to(device)

        positive_triples = torch.stack((sbj, rel, obj), dim=1)

        head_or_tail = torch.randint(high=2, size=sbj.size(), device=device)
        random_entities = torch.randint(high=len(entity2id), size=sbj.size(), device=device)
        neg_sbj = torch.where(head_or_tail == 1, random_entities, sbj)
        neg_obj = torch.where(head_or_tail == 0, random_entities, obj)
        negative_triples = torch.stack((neg_sbj, rel, neg_obj), dim=1)

        loss = model(positive_triples, negative_triples).mean()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    print('Epoch = {}, loss = {:.6f}'. format(epoch, loss))
    if(epoch % eval_freq == 0) :
        print('evaluation...')
        hit_at_10_score, mrr_score = evaluation(model, dev_loader, device)

        print('Dev set >> hit@10 : {:.2f}, mrr : {:.2f}'.format(hit_at_10_score, mrr_score))
        if(hit_at_10_score > best_score) :
            print('best model save...')
            state_dict = model.state_dict()

hit_at_10_score, mrr_score = evaluation(model, test_loader, device)


Epoch = 1, loss = 1.023869
Epoch = 2, loss = 0.913122
Epoch = 3, loss = 0.804188
Epoch = 4, loss = 0.836700
Epoch = 5, loss = 0.800225
Epoch = 6, loss = 0.735314
Epoch = 7, loss = 0.741244
Epoch = 8, loss = 0.722705
Epoch = 9, loss = 0.703607
Epoch = 10, loss = 0.667580
Epoch = 11, loss = 0.681190
Epoch = 12, loss = 0.637469
Epoch = 13, loss = 0.615523
Epoch = 14, loss = 0.600525
Epoch = 15, loss = 0.607204
Epoch = 16, loss = 0.588883
Epoch = 17, loss = 0.555680
Epoch = 18, loss = 0.589581
Epoch = 19, loss = 0.538721
Epoch = 20, loss = 0.547970
Epoch = 21, loss = 0.573983
Epoch = 22, loss = 0.556399
Epoch = 23, loss = 0.521527
Epoch = 24, loss = 0.526862
Epoch = 25, loss = 0.462608
evaluation...


RuntimeError: ignored

### 1.1 Вопрос о нормализации (2 балла)

Попробуйте обучить TransE без пятой строчки алгоритма (без нормализации по сущностям). Что происходит с обучением? Зачем требуется эта строка?

## 2. Нейросеть на гетерогенных данных (3 баллов)

Возьмите один из 2 датасетов (Freebase/ синтетический датасет hetero_graph далее)

In [ ]:
import numpy as np
import torch

n_users = 1000
n_items = 500
n_follows = 3000
n_clicks = 5000
n_dislikes = 500
n_hetero_features = 10
n_user_classes = 5
n_max_clicks = 10

follow_src = np.random.randint(0, n_users, n_follows)
follow_dst = np.random.randint(0, n_users, n_follows)
click_src = np.random.randint(0, n_users, n_clicks)
click_dst = np.random.randint(0, n_items, n_clicks)
dislike_src = np.random.randint(0, n_users, n_dislikes)
dislike_dst = np.random.randint(0, n_items, n_dislikes)

hetero_graph = dgl.heterograph({
    ('user', 'follow', 'user'): (follow_src, follow_dst),
    ('user', 'followed-by', 'user'): (follow_dst, follow_src),
    ('user', 'click', 'item'): (click_src, click_dst),
    ('item', 'clicked-by', 'user'): (click_dst, click_src),
    ('user', 'dislike', 'item'): (dislike_src, dislike_dst),
    ('item', 'disliked-by', 'user'): (dislike_dst, dislike_src)})

hetero_graph.nodes['user'].data['feature'] = torch.randn(n_users, n_hetero_features)
hetero_graph.nodes['item'].data['feature'] = torch.randn(n_items, n_hetero_features)
hetero_graph.nodes['user'].data['label'] = torch.randint(0, n_user_classes, (n_users,))
hetero_graph.edges['click'].data['label'] = torch.randint(1, n_max_clicks, (n_clicks,)).float()
# randomly generate training masks on user nodes and click edges
hetero_graph.nodes['user'].data['train_mask'] = torch.zeros(n_users, dtype=torch.bool).bernoulli(0.6)
hetero_graph.edges['click'].data['train_mask'] = torch.zeros(n_clicks, dtype=torch.bool).bernoulli(0.6)

Используя любую библиотеку (torch geometry, dgl, stellargraph) соберите нейронную сеть и обучите ее (решать задачу Node Classification) на одном из двух датасетов выше.

**БОНУСЫ:**
(2 балла) Обучите нейросеть решать задачу link prediction

(1 балл) возьмите еще какой-нибудь гетерогенный датасет (не маленький и не синтетический) и обучите на нем

(4 балла) реализуйте самостоятельно Relational GCN и продемонстрируйте работоспобность вашего слоя.